In [24]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_fscore_support
from sklearn.preprocessing import label_binarize
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from deap import base, creator, tools, algorithms

In [25]:
# 1. 데이터 불러오기
data = pd.read_csv('azdigar nav sartirovka+0 delete.csv')

In [26]:
# 분석할 열들
columns = [
    'w08chronic_a', 'w08chronic_b', 'w08chronic_c',
    'w08chronic_d', 'w08chronic_e', 'w08chronic_f',
    'w08chronic_g', 'w08chronic_h', 'w08chronic_i',
    'w08chronic_k', 'w08chronic_l', 'w08chronic_m'
]

In [27]:
# 2. 데이터 클리닝 - 'w08chronic_m'에서 값이 3인 행 제거
cleaned_data = data[data['w08chronic_m'] != 3]

In [28]:
# 3. 피처 데이터 정의 (모든 chronic 컬럼 제외)
X = cleaned_data.drop(columns=columns)
y_all = cleaned_data[columns]

In [29]:
# 4. 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [30]:
# 5. 모델 정의
models = {
    'Random Forest': RandomForestClassifier(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'MLP': MLPClassifier(max_iter=2000, random_state=42)
}


In [32]:
# 6. 유전 알고리즘을 위한 설정
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def evaluate(individual):
    selected_features = [i for i, bit in enumerate(individual) if bit]
    if len(selected_features) == 0:
        return (0,)  # 피처가 선택되지 않은 경우

    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    model = RandomForestClassifier(random_state=42)  # 여기에 평가할 모델을 지정할 수 있습니다.
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)

    return (f1_score(y_test, y_pred, average='weighted'),)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X_scaled.shape[1])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def run_ga(population_size=50, generations=10):
    population = toolbox.population(n=population_size)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=generations, verbose=True)
    best_individual = tools.selBest(population, 1)[0]
    return best_individual


In [33]:
# 7. 결과 저장을 위한 빈 데이터프레임 생성
result_table = pd.DataFrame(columns=['Target', 'Model', 'Precision', 'Recall', 'F1-Score', 'Accuracy', 'Micro F1', 'Macro F1', 'Weighted F1', 'AUC'])

In [34]:
# 8. 각 타겟 열을 사용한 학습 및 평가
for target_column in columns:
    print(f'\n[{target_column}] 열을 예측:')
    print('==================================')

    y = cleaned_data[target_column].values

    # 데이터 나누기
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # 9. 유전 알고리즘을 사용한 피처 선택
    best_individual = run_ga()
    selected_features = [i for i, bit in enumerate(best_individual) if bit]

    if len(selected_features) == 0:
        print(f'선택된 피처가 없습니다. {target_column} 열에 대한 평가를 건너뜁니다.')
        continue

    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    # 타겟 클래스가 두 개 이상이면 AUC 계산을 위해 타겟을 이진화 (One-vs-Rest 방식)
    y_test_bin = label_binarize(y_test, classes=np.unique(y))
    if y_test_bin.shape[1] == 1:  # 클래스가 하나인 경우 (로지스틱 회귀처럼 처리 안 되게)
        y_test_bin = np.concatenate([1 - y_test_bin, y_test_bin], axis=1)

    # 10. 각 모델에 대해 학습 및 평가
    for model_name, model in models.items():
        print(f'\n[{model_name}] 모델 평가:')
        print('--------------------------')

        try:
            # 모델 학습
            model.fit(X_train_selected, y_train)

            # 예측
            y_pred = model.predict(X_test_selected)
            
            # 확률 예측 값이 있는 경우
            if hasattr(model, "predict_proba"):
                y_pred_proba = model.predict_proba(X_test_selected)
            else:
                y_pred_proba = np.zeros((len(y_test), len(np.unique(y_test))))  # 확률 값을 지원하지 않는 모델에 대한 처리

            # 정확도 계산
            accuracy = accuracy_score(y_test, y_pred)

            # precision, recall, f1-score 계산 (weighted average)
            precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')

            # Micro, Macro, Weighted F1 계산
            micro_f1 = f1_score(y_test, y_pred, average='micro')
            macro_f1 = f1_score(y_test, y_pred, average='macro')
            weighted_f1 = f1_score(y_test, y_pred, average='weighted')

            # AUC 계산 (다중 클래스의 경우 One-vs-Rest 방식 사용)
            try:
                auc = roc_auc_score(y_test_bin, y_pred_proba, multi_class="ovr", average='weighted')
            except ValueError:
                auc = np.nan  # AUC 계산 불가 시 처리

            # 결과 추가
            new_row = pd.DataFrame({
                'Model': [model_name],
                'Target': [target_column],
                'Precision': [precision],
                'Recall': [recall],
                'F1-Score': [f1],
                'Accuracy': [accuracy],
                'Micro F1': [micro_f1],
                'Macro F1': [macro_f1],
                'Weighted F1': [weighted_f1],
                'AUC': [auc]
            })
            result_table = pd.concat([result_table, new_row], ignore_index=True)
        except Exception as e:
            print(f"{model_name} 모델에서 오류 발생: {e}")
            continue


[w08chronic_a] 열을 예측:
gen	nevals
0  	50    
1  	32    
2  	36    
3  	25    
4  	28    
5  	30    
6  	26    
7  	27    
8  	25    
9  	27    
10 	26    

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------

[MLP] 모델 평가:
--------------------------

[w08chronic_b] 열을 예측:
gen	nevals
0  	50    
1  	36    
2  	27    
3  	30    
4  	27    
5  	31    
6  	26    
7  	33    
8  	23    
9  	35    
10 	21    

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[MLP] 모델 평가:
--------------------------

[w08chronic_c] 열을 예측:
gen	nevals
0  	50    
1  	28    
2  	29    
3  	22    
4  	35    
5  	33    
6  	27    
7  	26    
8  	33    
9  	23    
10 	25    

[Random Forest] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------


...


[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[MLP] 모델 평가:
--------------------------

[w08chronic_m] 열을 예측:
gen	nevals
0  	50    
1  	38    
2  	25    
3  	31    
4  	32    
5  	31    
6  	35    
7  	27    
8  	32    
9  	22    
10 	26    

[Random Forest] 모델 평가:
--------------------------

[Decision Tree] 모델 평가:
--------------------------

[SVM] 모델 평가:
--------------------------


c:\Users\azamj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



[MLP] 모델 평가:
--------------------------


In [35]:
# 10. 결과 테이블 출력
print("\n전체 모델 평가 결과:")
print(result_table)


전체 모델 평가 결과:
          Target          Model  Precision    Recall  F1-Score  Accuracy  \
0   w08chronic_a  Random Forest   0.765374  0.746137  0.743510  0.746137   
1   w08chronic_a  Decision Tree   0.676606  0.675497  0.675653  0.675497   
2   w08chronic_a            SVM   0.757848  0.718543  0.710913  0.718543   
3   w08chronic_a            MLP   0.682727  0.682119  0.682260  0.682119   
4   w08chronic_b  Random Forest   0.723311  0.771523  0.694203  0.771523   
5   w08chronic_b  Decision Tree   0.668346  0.665563  0.666938  0.665563   
6   w08chronic_b            SVM   0.591847  0.769316  0.669012  0.769316   
7   w08chronic_b            MLP   0.685979  0.700883  0.692781  0.700883   
8   w08chronic_c  Random Forest   0.859611  0.927152  0.892105  0.927152   
9   w08chronic_c  Decision Tree   0.876913  0.870861  0.873842  0.870861   
10  w08chronic_c            SVM   0.859611  0.927152  0.892105  0.927152   
11  w08chronic_c            MLP   0.877986  0.901766  0.888770  0.901766  